#### Mattie Gisselbeck
#### GIS 5571: ArcGIS I 
#### November 2, 2022
#### Lab 2 - Part 1.3

#### 1.3.1: Downloads the Annual 30-Year Normal .bil files from PRISM

In [7]:
# Import Packages
import arcpy
import os
import requests
from zipfile import ZipFile

In [9]:
# Setting File Paths for Outputs
zip_path = os.path.join(os.getcwd(), "data/PRISM")
file_name = os.path.join(zip_path, "PRISM_ppt_30yr_normal_4kmM3_all_bil.zip")

# Check if Path Exists
if os.path.exists(zip_path) == False:
    os.mkdir(zip_path)

prism_url = "https://ftp.prism.oregonstate.edu/normals_4km/ppt/PRISM_ppt_30yr_normal_4kmM3_all_bil.zip"

resp = requests.get(prism_url)

if os.path.exists(file_name) == False:
    with open(file_name, "wb") as z:
        z.write(resp.content)
else:
    print("ZIP file already exists.")

# Unzipping File
if os.path.exists(file_name[:-3]) == False:
    with ZipFile(file_name, "r") as zipped:
        zipped.extractall(file_name[:-3])
else:
    print("File has already been unzipped.")

#### 1.3.2: Converts the Annual 30-Year .bil data into a Space Time Cube and Exports it to a Disk

In [12]:
# Create Mosaic Dataset
sr = arcpy.SpatialReference(3857)
empty_mosaic = arcpy.management.CreateMosaicDataset(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb", "PRISM_ALL_1_12", sr)

In [13]:
# Add Rasters to Mosaic Dataset
arcpy.management.AddRastersToMosaicDataset("PRISM_ALL_1_12", "Raster Dataset", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_01_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_02_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_03_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_04_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_05_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_06_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_07_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_08_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_09_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_10_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_11_bil.bil;C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_ppt_30yr_normal_4kmM3_12_bil.bil", "UPDATE_CELL_SIZES", "UPDATE_BOUNDARY", "NO_OVERVIEWS", None, 0, 1500, None, '', "SUBFOLDERS", "ALLOW_DUPLICATES", "BUILD_PYRAMIDS", "CALCULATE_STATISTICS", "NO_THUMBNAILS", '', "NO_FORCE_SPATIAL_REFERENCE", "ESTIMATE_STATISTICS", None, "NO_PIXEL_CACHE", r"C:\Users\gisse015\AppData\Local\ESRI\rasterproxies\PRISM_ALL_1_12")

<Result 'PRISM_ALL_1_12'>

In [17]:
# Calculate Field for Variable and Timestamp
arcpy.management.CalculateField(r"PRISM_ALL_1_12\Footprint", "Variable", '"PPT"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(r"PRISM_ALL_1_12", "Timestamp", 'DateAdd(Date(2021, 0, 1), $feature.OBJECTID-1, "month")', "ARCADE", '', "DATE", "NO_ENFORCE_DOMAINS")

<Result 'PRISM_ALL_1_12'>

In [25]:
# Build Multidimensional Info
arcpy.md.BuildMultidimensionalInfo("PRISM_ALL_1_12", "Variable", "Timestamp # #", "PPT # #", "NO_DELETE_MULTIDIMENSIONAL_INFO")

<Result 'PRISM_ALL_1_12'>

In [27]:
# Make Multidimensional Raster Later
arcpy.md.MakeMultidimensionalRasterLayer("PRISM_ALL_1_12", "PRISM_ALL_1_12_MultidimLayer1", "PPT", "ALL", None, None, '', '', '', None, '', "-13917257.2777 2761917.52169259 -7400428.75412497 6435460.7179", "DIMENSIONS")

<Result 'PRISM_ALL_1_12_MultidimLayer1'>

In [28]:
# Create Space Time Cube from Multidimensional Raster Layer
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer("PRISM_ALL_1_12_MultidimLayer1", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_STC.nc", "ZEROS")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\PRISM_STC.nc'>

#### 1.3.3: Exports an Animation of the Timeseries

In [29]:
# Visualize Space Time Cube for Animation 
arcpy.stpm.VisualizeSpaceTimeCube3D(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\PRISM_STC.nc", "PPT_NONE_ZEROS", "VALUE", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\PRISM_STC_VisualizeSpaceTimeCube3D")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\PRISM_STC_VisualizeSpaceTimeCube3D'>